*This Notebook provides a way to get rid of the data batch-effect. As a result of the work, a gene usage tables are formed. The [source code](https://github.com/antigenomics/mirpy) was kindly provided by our supervisors [Mikhail Shugai](https://github.com/mikessh) and [Elizaveta Vlasova](https://github.com/VEK239).*

In [1]:
import sys
import os
import re
import time

sys.path.append("../")

from mir.common.parser import *
from mir.common.repertoire import Repertoire
from mir.common.repertoire_dataset import RepertoireDataset
from mir.basic.segment_usage import *
from mir.basic.sampling import RepertoireSampling

# Cohort-I
*fmba, Russia*

## Alpha chain

In [2]:
#metadata
metadata_path = '../../../data/cohort_I/metadata/metadata_fmba_full.txt'
batch_meta_path = '../../../data/cohort_I/metadata/desc_fmba_not_nan_hla.csv'

metadata = pd.read_csv(metadata_path, sep='\t')
metadata.rename(columns={'id': 'file.id_short'}, inplace=True)
metadata['file.id_short'] = metadata['file.id_short'].astype(int)

batch_meta = pd.read_csv(batch_meta_path, sep=',')
batch_meta['file.id_short'] = batch_meta['file.id'].str.split('_').str[0]
batch_meta['file.id_short'] = batch_meta['file.id_short'].astype(int)
batch_meta['run'] = batch_meta['file.id'] + '.clonotypes.TRA.txt'

new_meta = pd.merge(metadata, batch_meta[['file.id_short', 'run', 'folder']], on='file.id_short')
new_meta = new_meta[new_meta['sample.COVID_status'] == 'healthy']
new_meta = new_meta.reset_index(drop=True)
new_meta

,file.id_short,sample.age,sample.sex,sample.height(cm),sample.weight(kg),sample.waist(cm),sample.diastolicBloodPressure(mmHg),sample.bodyTemperature,sample.pulse(beats/min),sample.oxygenLevel(%),...,sample.HLA-DRB1.1,sample.HLA-DRB1.2,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,run,folder
0,20000200808,19,male,187.0,105.0,97.0,80,36.6,74,99.0,...,DRB1*01:01,DRB1*04:04,NaN,NaN,NaN,NaN,NaN,NaN,020000200808_S181_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5
1,20000350808,54,female,159.0,86.0,96.0,90,36.3,60,98.0,...,DRB1*07:01,DRB1*07:01,NaN,NaN,NaN,NaN,NaN,NaN,020000350808_S126_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5
2,20000360808,30,male,186.0,86.0,89.0,80,36.4,80,97.0,...,DRB1*03:01,DRB1*07:01,NaN,NaN,NaN,NaN,NaN,NaN,020000360808_S102_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5
3,20000390808,33,female,157.0,65.0,77.0,60,36.6,72,97.0,...,DRB1*03:01,DRB1*12:02,NaN,NaN,NaN,NaN,NaN,NaN,020000390808_S189_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5
4,20000640808,68,female,161.0,103.0,115.0,80,36.6,75,99.0,...,DRB1*01:01,DRB1*15:01,NaN,NaN,NaN,NaN,NaN,NaN,020000640808_S117_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,780003510808,61,male,176.0,85.0,106.0,80,36.4,60,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003510808_S92_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4
233,780003520808,81,male,172.0,74.0,96.0,78,36.5,70,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003520808_S12_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4
234,780003890808,27,female,168.0,61.0,59.0,70,36.5,64,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003890808_S35_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4
235,780003930808,64,male,171.0,60.0,89.0,75,36.6,72,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003930808_S76_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4


In [3]:
base_path = '../../../data/cohort_I/data/'
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=new_meta,
                                 threads=28,
                                 gene = 'TRA',
                                 paths=[base_path + r['run']
                                        for _, r in new_meta.iterrows()])
print(time.time() - t0)

3574192
500.78504848480225


In [4]:
dataset.metadata

,file.id_short,sample.age,sample.sex,sample.height(cm),sample.weight(kg),sample.waist(cm),sample.diastolicBloodPressure(mmHg),sample.bodyTemperature,sample.pulse(beats/min),sample.oxygenLevel(%),...,sample.HLA-DRB1.2,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,run,folder,path
0,20000200808,19,male,187.0,105.0,97.0,80,36.6,74,99.0,...,DRB1*04:04,NaN,NaN,NaN,NaN,NaN,NaN,020000200808_S181_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/data/020000200808_S181_...
1,20000350808,54,female,159.0,86.0,96.0,90,36.3,60,98.0,...,DRB1*07:01,NaN,NaN,NaN,NaN,NaN,NaN,020000350808_S126_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/data/020000350808_S126_...
2,20000360808,30,male,186.0,86.0,89.0,80,36.4,80,97.0,...,DRB1*07:01,NaN,NaN,NaN,NaN,NaN,NaN,020000360808_S102_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/data/020000360808_S102_...
3,20000390808,33,female,157.0,65.0,77.0,60,36.6,72,97.0,...,DRB1*12:02,NaN,NaN,NaN,NaN,NaN,NaN,020000390808_S189_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/data/020000390808_S189_...
4,20000640808,68,female,161.0,103.0,115.0,80,36.6,75,99.0,...,DRB1*15:01,NaN,NaN,NaN,NaN,NaN,NaN,020000640808_S117_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/data/020000640808_S117_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,780003510808,61,male,176.0,85.0,106.0,80,36.4,60,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003510808_S92_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/data/780003510808_S92_L...
233,780003520808,81,male,172.0,74.0,96.0,78,36.5,70,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003520808_S12_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/data/780003520808_S12_L...
234,780003890808,27,female,168.0,61.0,59.0,70,36.5,64,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003890808_S35_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/data/780003890808_S35_L...
235,780003930808,64,male,171.0,60.0,89.0,75,36.6,72,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003930808_S76_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/data/780003930808_S76_L...


In [5]:
dataset.evaluate_segment_usage()

,TRAV35*01,TRAJ43*01,TRAJ8*01,TRAJ26*01,TRAJ25*01,TRAV8-6*01,TRAJ27*01,TRAJ19*01,TRAV9-2*01,TRAV1-1*01,...,TRAV17*01,TRAJ6*01,TRAJ21*01,TRAV19*01,TRAV18*01,TRAV12-1*01,TRAV9-1*01,TRAV26-1*01,TRAJ9*01,TRAJ61*01
0,1112,1277,197,27,19,1006,557,0,1664,330,...,1463,624,3,1409,1,1336,17,1559,554,537
1,791,500,282,27,13,833,493,0,1613,342,...,1292,628,2,996,1,1186,7,1250,608,276
2,1152,887,226,25,9,1017,578,0,1682,383,...,1532,821,2,1266,1,1481,15,1271,790,475
3,857,706,203,25,15,701,419,1,1164,356,...,1257,479,1,843,0,950,9,874,447,453
4,72,64,15,4,2,67,40,0,115,34,...,135,36,0,60,0,97,1,107,56,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,97,67,19,4,2,150,93,0,256,81,...,242,118,0,137,0,216,2,161,108,41
233,489,692,113,5,5,613,261,0,990,191,...,941,432,1,521,0,662,0,930,368,165
234,131,118,30,6,2,132,61,0,247,38,...,230,80,0,164,0,177,1,151,80,64
235,261,175,29,1,0,236,142,0,445,86,...,371,186,0,238,0,325,1,439,170,69


In [6]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [7]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')

In [8]:
norm_usage_table_v

     TRAV35*01  TRAV8-6*01  TRAV9-2*01  TRAV1-1*01  TRAV22*01  TRAV36DV7*01  \
0     0.029492    0.026681    0.044132    0.008752   0.020130      0.015515   
1     0.024952    0.026277    0.050882    0.010788   0.016813      0.012713   
2     0.029591    0.026123    0.043205    0.009838   0.019496      0.014513   
3     0.029603    0.024214    0.040207    0.012297   0.020276      0.013264   
4     0.027896    0.025959    0.044556    0.013173   0.019760      0.014336   
..         ...         ...         ...         ...        ...           ...   
232   0.018979    0.029348    0.050088    0.015848   0.016239      0.009783   
233   0.025247    0.031649    0.051113    0.009861   0.017451      0.012597   
234   0.026039    0.026237    0.049096    0.007553   0.016895      0.014311   
235   0.030877    0.027919    0.052644    0.010174   0.024015      0.003076   
236   0.023387    0.020866    0.049817    0.009824   0.018779      0.017649   

     TRAV13-1*01  TRAV6*01  TRAV8-4*01  TRAV21*01  

#### Sampling reads from the repertoire, not changing the gene usage distribution

In [9]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [10]:
sampled_repertoire

Repertoire of 21868 clonotypes and 100000 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVRDG*_QGGKLIF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAATDSWGKLQF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSPLW_NNRLAF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASVDAGNMLTF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CATGPTR_GTALIF cdr3nt_4
file.id_short                                                                20000200808
sample.age                                                                            19
sample.sex                

#### Resampling the repertoire using the new derived usages for V and J genes

In [11]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

6.547260761260986


In [12]:
resampled_repertoire

Repertoire of 36002 clonotypes and 9537751 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVRDG*_QGGKLIF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAATDSWGKLQF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CATGPTR_GTALIF cdr3nt_4
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSPLW_NNRLAF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVSDGWGNTPLVF cdr3nt_10
file.id_short                                                                20000200808
sample.age                                                                            19
sample.sex            

#### Resampling all the reperoires from a dataset

In [13]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)

3271399
563.537807226181


In [14]:
dataset[14]

Repertoire of 29777 clonotypes and 6356503 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAGQRFGGSQGNLIF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAGQTFYNQGGKLIF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CLLGDPGGTSYGKLTF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVSLQGGKLIF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAETPIYNQGGKLIF cdr3nt_4
file.id_short                                                                50001340808
sample.age                                                                            28
sample.sex         

In [15]:
resampled_dataset[14]

Repertoire of 28822 clonotypes and 6356503 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CLLGDPGGTSYGKLTF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVFWGFNKFYF cdr3nt_349
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVNAPSNARLMF cdr3nt_5
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSAS_DKFYF cdr3nt_410
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSPYNFNKFYF cdr3nt_700
file.id_short                                                                50001340808
sample.age                                                                            28
sample.sex          

In [16]:
trav_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRA', segment_type='V').segment_usage_matrix

In [17]:
traj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRA', segment_type='J').segment_usage_matrix

In [18]:
traj_matrix

,TRAJ43*01,TRAJ8*01,TRAJ26*01,TRAJ25*01,TRAJ27*01,TRAJ19*01,TRAJ29*01,TRAJ28*01,TRAJ38*01,TRAJ18*01,...,TRAJ14*01,TRAJ40*01,TRAJ15*01,TRAJ41*01,TRAJ13*01,TRAJ57*01,TRAJ6*01,TRAJ21*01,TRAJ9*01,TRAJ61*01
0,0.033915,0.005055,0.000750,0.000389,0.015110,0.000000,0.024582,0.024804,0.010638,0.009111,...,0.000028,0.031332,0.018110,0.016971,0.017055,0.021693,0.016777,0.0,0.015249,0.014221
1,0.016807,0.008690,0.000808,0.000337,0.015999,0.000000,0.028461,0.023206,0.009296,0.009700,...,0.000168,0.027854,0.016908,0.013675,0.019434,0.022465,0.019906,0.0,0.019299,0.008454
2,0.023339,0.005713,0.000647,0.000189,0.015254,0.000000,0.030831,0.023500,0.007681,0.009756,...,0.000027,0.030534,0.017652,0.016817,0.018730,0.009405,0.021344,0.0,0.020428,0.012181
3,0.025046,0.006808,0.000896,0.000466,0.014798,0.000036,0.026766,0.026479,0.011036,0.008922,...,0.000036,0.029811,0.015945,0.016733,0.018095,0.013042,0.016769,0.0,0.015766,0.015551
4,0.025530,0.005193,0.001731,0.000865,0.015578,0.000000,0.026828,0.023367,0.007356,0.007356,...,0.000000,0.020770,0.023367,0.014712,0.025963,0.000865,0.015145,0.0,0.020770,0.005193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,0.012786,0.003773,0.000838,0.000419,0.019074,0.000000,0.033117,0.024942,0.005869,0.011318,...,0.000000,0.022427,0.019493,0.015720,0.021798,0.006078,0.024104,0.0,0.021798,0.008384
233,0.036421,0.005876,0.000212,0.000159,0.013393,0.000000,0.025040,0.021440,0.009317,0.009052,...,0.000053,0.034569,0.017046,0.018317,0.020487,0.015352,0.022340,0.0,0.019111,0.008576
234,0.023800,0.005647,0.001210,0.000403,0.012303,0.000000,0.026624,0.021178,0.012505,0.010085,...,0.000202,0.032271,0.019968,0.015329,0.017346,0.023598,0.016136,0.0,0.016136,0.012707
235,0.021071,0.003512,0.000125,0.000000,0.016681,0.000000,0.025712,0.028722,0.003386,0.010410,...,0.000000,0.021824,0.021197,0.015552,0.019691,0.006522,0.022451,0.0,0.019691,0.008153


In [19]:
trav_matrix.to_csv('../../../data/cohort_I/data_corrected/v_clonotypes_TRA.csv', index = False)
traj_matrix.to_csv('../../../data/cohort_I/data_corrected/j_clonotypes_TRA.csv', index = False)

### Nonfunctional sequences

In [21]:
base_path = '../../../data/cohort_I/nonfunctional/'
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=new_meta,
                                 threads=28,
                                 gene = 'TRA',
                                 paths=[base_path + r['run']
                                        for _, r in new_meta.iterrows()])
print(time.time() - t0)

0
285.883665561676


In [22]:
dataset.metadata

,file.id_short,sample.age,sample.sex,sample.height(cm),sample.weight(kg),sample.waist(cm),sample.diastolicBloodPressure(mmHg),sample.bodyTemperature,sample.pulse(beats/min),sample.oxygenLevel(%),...,sample.HLA-DRB1.2,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,run,folder,path
0,20000200808,19,male,187.0,105.0,97.0,80,36.6,74,99.0,...,DRB1*04:04,NaN,NaN,NaN,NaN,NaN,NaN,020000200808_S181_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/nonfunctional/020000200...
1,20000350808,54,female,159.0,86.0,96.0,90,36.3,60,98.0,...,DRB1*07:01,NaN,NaN,NaN,NaN,NaN,NaN,020000350808_S126_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/nonfunctional/020000350...
2,20000360808,30,male,186.0,86.0,89.0,80,36.4,80,97.0,...,DRB1*07:01,NaN,NaN,NaN,NaN,NaN,NaN,020000360808_S102_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/nonfunctional/020000360...
3,20000390808,33,female,157.0,65.0,77.0,60,36.6,72,97.0,...,DRB1*12:02,NaN,NaN,NaN,NaN,NaN,NaN,020000390808_S189_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/nonfunctional/020000390...
4,20000640808,68,female,161.0,103.0,115.0,80,36.6,75,99.0,...,DRB1*15:01,NaN,NaN,NaN,NaN,NaN,NaN,020000640808_S117_L002.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq5,../../../data/cohort_I/nonfunctional/020000640...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,780003510808,61,male,176.0,85.0,106.0,80,36.4,60,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003510808_S92_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/nonfunctional/780003510...
233,780003520808,81,male,172.0,74.0,96.0,78,36.5,70,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003520808_S12_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/nonfunctional/780003520...
234,780003890808,27,female,168.0,61.0,59.0,70,36.5,64,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003890808_S35_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/nonfunctional/780003890...
235,780003930808,64,male,171.0,60.0,89.0,75,36.6,72,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780003930808_S76_L001.clonotypes.TRA.txt,2020/10_FMBA_NovaSeq4,../../../data/cohort_I/nonfunctional/780003930...


In [23]:
dataset.evaluate_segment_usage()

,TRAV35*01,TRAJ43*01,TRAJ8*01,TRAJ26*01,TRAJ25*01,TRAV8-6*01,TRAJ27*01,TRAJ19*01,TRAV9-2*01,TRAV1-1*01,...,TRAV17*01,TRAJ6*01,TRAJ21*01,TRAV19*01,TRAV18*01,TRAV12-1*01,TRAV9-1*01,TRAV26-1*01,TRAJ9*01,TRAJ61*01
0,634,615,61,12,12,377,269,0,663,178,...,785,278,1,634,0,646,8,632,187,369
1,361,216,103,9,5,278,213,0,479,169,...,537,233,0,546,0,537,4,498,166,180
2,605,470,82,8,4,382,279,0,671,227,...,835,350,1,646,1,685,10,633,268,306
3,466,382,89,7,7,303,213,1,499,162,...,626,212,1,495,0,498,7,462,157,291
4,32,24,4,0,0,20,13,0,35,21,...,55,16,0,16,0,43,1,44,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,52,28,8,1,1,34,40,0,91,35,...,106,54,0,74,0,77,2,71,37,24
233,270,294,39,1,2,177,106,0,321,80,...,356,183,0,231,0,269,0,353,96,109
234,64,61,12,3,2,49,29,0,96,16,...,99,34,0,81,0,78,1,78,20,38
235,114,71,6,0,0,65,53,0,157,25,...,148,89,0,108,0,119,1,155,45,56


In [24]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [25]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')

In [26]:
norm_usage_table_v

     TRAV35*01  TRAV8-6*01  TRAV9-2*01  TRAV1-1*01  TRAV22*01  TRAV36DV7*01  \
0     0.035115    0.020881    0.036721    0.009859   0.019662      0.017945   
1     0.026544    0.020441    0.035221    0.012426   0.018971      0.013529   
2     0.031757    0.020051    0.035221    0.011915   0.021416      0.015485   
3     0.032138    0.020897    0.034414    0.011172   0.022966      0.014414   
4     0.030740    0.019212    0.033622    0.020173   0.024015      0.016330   
..         ...         ...         ...         ...        ...           ...   
232   0.024857    0.016252    0.043499    0.016730   0.016252      0.009082   
233   0.033826    0.022175    0.040215    0.010023   0.020296      0.013781   
234   0.028046    0.021472    0.042068    0.007011   0.015337      0.013146   
235   0.033918    0.019339    0.046712    0.007438   0.019637      0.002380   
236   0.028844    0.019097    0.036204    0.009946   0.019296      0.018301   

     TRAV13-1*01  TRAV6*01  TRAV8-4*01  TRAV21*01  

#### Sampling reads from the repertoire, not changing the gene usage distribution

In [27]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [28]:
sampled_repertoire

Repertoire of 12315 clonotypes and 100000 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVRDG*_QGGKLIF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSPLW_NNRLAF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CATGPTR_GTALIF cdr3nt_4
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAGVPRT_GNQFYF cdr3nt_5
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CIVRLVP_VNRKLTF cdr3nt_6
file.id_short                                                                20000200808
sample.age                                                                            19
sample.sex           

#### Resampling the repertoire using the new derived usages for V and J genes

In [29]:
dataset[0]

Repertoire of 18055 clonotypes and 4514408 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVRDG*_QGGKLIF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSPLW_NNRLAF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CATGPTR_GTALIF cdr3nt_4
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAGVPRT_GNQFYF cdr3nt_5
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CIVRLVP_VNRKLTF cdr3nt_6
file.id_short                                                                20000200808
sample.age                                                                            19
sample.sex          

In [30]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

2.8143439292907715


In [31]:
resampled_repertoire

Repertoire of 17238 clonotypes and 4514408 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVRDG*_QGGKLIF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSPLW_NNRLAF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CATGPTR_GTALIF cdr3nt_4
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVEW_QKLLF cdr3nt_8
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAFRMM_GFKTIF cdr3nt_14
file.id_short                                                                20000200808
sample.age                                                                            19
sample.sex              

#### Resampling all the reperoires from a dataset

In [32]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)

0
248.3766894340515


In [33]:
dataset[14]

Repertoire of 15997 clonotypes and 3186951 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAGLKRG_AQKLVF cdr3nt_8
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASQICS_GNNRLAF cdr3nt_9
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CIVIRLCG_TGNQFYF cdr3nt_10
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CIPL_NMRF cdr3nt_12
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVSPCM_NNRLAF cdr3nt_14
file.id_short                                                                50001340808
sample.age                                                                            28
sample.sex           

In [34]:
resampled_dataset[14]

Repertoire of 15466 clonotypes and 3186951 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVLQLQQIFQ*VGNFNKFYF cdr3nt_3958
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CIVIRLCG_TGNQFYF cdr3nt_10
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAT*L_NKFYF cdr3nt_2072
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAVDTT_NKFYF cdr3nt_324
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CVVSAS_DKFYF cdr3nt_410
file.id_short                                                                50001340808
sample.age                                                                            28
sample.se

In [35]:
nf_trav_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRA', segment_type='V').segment_usage_matrix

In [36]:
nf_traj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRA', segment_type='J').segment_usage_matrix

In [37]:
nf_traj_matrix

,TRAJ43*01,TRAJ8*01,TRAJ26*01,TRAJ25*01,TRAJ27*01,TRAJ19*01,TRAJ29*01,TRAJ28*01,TRAJ38*01,TRAJ18*01,...,TRAJ14*01,TRAJ40*01,TRAJ15*01,TRAJ41*01,TRAJ13*01,TRAJ57*01,TRAJ6*01,TRAJ21*01,TRAJ9*01,TRAJ61*01
0,0.033821,0.003307,0.000638,0.000464,0.015257,0.000000,0.023611,0.025525,0.013343,0.008992,...,0.000000,0.028310,0.015199,0.015431,0.013807,0.024249,0.015605,0.0,0.010790,0.020362
1,0.016863,0.007651,0.000703,0.000156,0.015926,0.000000,0.028339,0.023187,0.010227,0.008666,...,0.000468,0.022953,0.014521,0.013740,0.015380,0.022875,0.017332,0.0,0.012491,0.013428
2,0.025352,0.004253,0.000331,0.000166,0.014637,0.000000,0.030544,0.023364,0.010052,0.008893,...,0.000000,0.029716,0.016073,0.016570,0.017012,0.010218,0.018835,0.0,0.014195,0.015742
3,0.027033,0.005967,0.000431,0.000216,0.014954,0.000072,0.026026,0.025883,0.013804,0.010497,...,0.000144,0.026242,0.014092,0.015601,0.015242,0.014451,0.014954,0.0,0.010928,0.019915
4,0.024803,0.004510,0.000000,0.000000,0.013529,0.000000,0.024803,0.041714,0.012401,0.009019,...,0.000000,0.019166,0.027057,0.011274,0.021421,0.001127,0.016911,0.0,0.014656,0.007892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,0.013804,0.003579,0.000511,0.000000,0.019427,0.000000,0.037832,0.027607,0.007669,0.012270,...,0.000000,0.018916,0.016360,0.016360,0.021472,0.004601,0.025562,0.0,0.018916,0.012270
233,0.037503,0.005009,0.000128,0.000257,0.013486,0.000000,0.028127,0.022219,0.011816,0.009761,...,0.000000,0.029926,0.014385,0.020293,0.018366,0.015155,0.022990,0.0,0.012073,0.013743
234,0.027099,0.005331,0.001333,0.000888,0.011995,0.000000,0.020435,0.025766,0.019991,0.009329,...,0.000444,0.025766,0.020435,0.010662,0.016881,0.022657,0.015104,0.0,0.008885,0.016437
235,0.022486,0.001606,0.000000,0.000000,0.015740,0.000000,0.021844,0.033730,0.003855,0.011564,...,0.000000,0.020238,0.019274,0.018632,0.017347,0.008352,0.025699,0.0,0.014134,0.017668


In [38]:
nf_trav_matrix.to_csv('../../../data/cohort_I/nonfunctional_corrected/nf_v_clonotypes_TRA.csv', index = False)
nf_traj_matrix.to_csv('../../../data/cohort_I/nonfunctional_corrected/nf_j_clonotypes_TRA.csv', index = False)

### Onlyfunctional sequences

In [ ]:
base_path = '../../../data/cohort_I/onlyfunctional/'
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=new_meta,
                                 threads=28,
                                 gene = 'TRA',
                                 paths=[base_path + r['run']
                                        for _, r in new_meta.iterrows()])
print(time.time() - t0)

3574192


In [ ]:
dataset.metadata

In [ ]:
dataset.evaluate_segment_usage()

In [ ]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [ ]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRA', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')


In [ ]:
dataset[0].number_of_reads

In [ ]:
norm_usage_table_v

#### Sampling reads from the repertoire, not changing the gene usage distribution

In [ ]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [ ]:
sampled_repertoire

#### Resampling the repertoire using the new derived usages for V and J genes

In [ ]:
dataset[0]

In [ ]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

In [ ]:
resampled_repertoire

#### Resampling all the reperoires from a dataset

In [ ]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)

In [ ]:
dataset[14]

In [ ]:
resampled_dataset[14]

In [ ]:
of_trav_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRA', segment_type='V').segment_usage_matrix

In [ ]:
of_traj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRA', segment_type='J').segment_usage_matrix

In [ ]:
of_traj_matrix

In [ ]:
of_trav_matrix.to_csv('../../../data/cohort_I/onlyfunctional_corrected/of_v_clonotypes_TRA.csv', index = False)
of_traj_matrix.to_csv('../../../data/cohort_I/onlyfunctional_corrected/of_j_clonotypes_TRA.csv', index = False)

## Beta chain

In [ ]:
#metadata
metadata_path = '../../../data/cohort_I/metadata/metadata_fmba_full.txt'
batch_meta_path = '../../../data/cohort_I/metadata/desc_fmba_not_nan_hla.csv'

metadata = pd.read_csv(metadata_path, sep='\t')
metadata.rename(columns={'id': 'file.id_short'}, inplace=True)
metadata['file.id_short'] = metadata['file.id_short'].astype(int)

batch_meta = pd.read_csv(batch_meta_path, sep=',')
batch_meta['file.id_short'] = batch_meta['file.id'].str.split('_').str[0]
batch_meta['file.id_short'] = batch_meta['file.id_short'].astype(int)
batch_meta['run'] = batch_meta['file.id'] + '.clonotypes.TRB.txt'

new_meta = pd.merge(metadata, batch_meta[['file.id_short', 'run', 'folder']], on='file.id_short')
new_meta = new_meta[new_meta['sample.COVID_status'] == 'healthy']
new_meta = new_meta.reset_index(drop=True)
new_meta

In [ ]:
base_path = '../../../data/cohort_I/data/'
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=new_meta,
                                 threads=28,
                                 gene = 'TRB',
                                 paths=[base_path + r['run']
                                        for _, r in new_meta.iterrows()])
print(time.time() - t0)

In [ ]:
dataset.metadata

In [ ]:
dataset.evaluate_segment_usage()

In [ ]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [ ]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')

In [ ]:
norm_usage_table_v

#### Sampling reads from the repertoire, not changing the gene usage distribution

In [ ]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [ ]:
sampled_repertoire

#### Resampling the repertoire using the new derived usages for V and J genes

In [ ]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

In [ ]:
resampled_repertoire

#### Resampling all the reperoires from a dataset

In [ ]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)

In [ ]:
dataset[14]

In [ ]:
resampled_dataset[14]

In [ ]:
trbv_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='V').segment_usage_matrix

In [ ]:
trBj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='J').segment_usage_matrix

In [ ]:
trbj_matrix

In [ ]:
trbv_matrix.to_csv('../../../data/cohort_I/data_corrected/v_clonotypes_TRB.csv', index = False)
trbj_matrix.to_csv('../../../data/cohort_I/data_corrected/j_clonotypes_TRB.csv', index = False)

### Nonfunctional sequences

In [ ]:
base_path = '../../../data/cohort_I/nonfunctional/'
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=new_meta,
                                 threads=28,
                                 gene = 'TRB',
                                 paths=[base_path + r['run']
                                        for _, r in new_meta.iterrows()])
print(time.time() - t0)

In [ ]:
dataset.metadata

In [ ]:
dataset.evaluate_segment_usage()

In [ ]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [ ]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')

In [ ]:
norm_usage_table_v

#### Sampling reads from the repertoire, not changing the gene usage distribution

In [ ]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [ ]:
sampled_repertoire

#### Resampling the repertoire using the new derived usages for V and J genes

In [ ]:
dataset[0]

In [ ]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

In [ ]:
resampled_repertoire

#### Resampling all the reperoires from a dataset

In [ ]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)

In [ ]:
dataset[14]

In [ ]:
resampled_dataset[14]

In [ ]:
nf_trbv_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='V').segment_usage_matrix

In [ ]:
nf_trbj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='J').segment_usage_matrix

In [ ]:
nf_trbj_matrix

In [ ]:
nf_trbv_matrix.to_csv('../../../data/cohort_I/nonfunctional_corrected/nf_v_clonotypes_TRB.csv', index = False)
nf_trbj_matrix.to_csv('../../../data/cohort_I/nonfunctional_corrected/nf_j_clonotypes_TRB.csv', index = False)

### Onlyfunctional sequences

In [ ]:
base_path = '../../../data/cohort_I/onlyfunctional/'
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=new_meta,
                                 threads=28,
                                 gene = 'TRB',
                                 paths=[base_path + r['run']
                                        for _, r in new_meta.iterrows()])
print(time.time() - t0)

In [ ]:
dataset.metadata

In [ ]:
dataset.evaluate_segment_usage()

In [ ]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [ ]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')


In [ ]:
dataset[0].number_of_reads

In [ ]:
norm_usage_table_v

#### Sampling reads from the repertoire, not changing the gene usage distribution

In [ ]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [ ]:
sampled_repertoire

#### Resampling the repertoire using the new derived usages for V and J genes

In [ ]:
dataset[0]

In [ ]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

In [ ]:
resampled_repertoire

#### Resampling all the reperoires from a dataset

In [ ]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)

In [ ]:
dataset[14]

In [ ]:
resampled_dataset[14]

In [ ]:
of_trbv_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='V').segment_usage_matrix

In [ ]:
of_trbj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='J').segment_usage_matrix

In [ ]:
of_trbj_matrix

In [ ]:
of_trbv_matrix.to_csv('../../../data/cohort_I/onlyfunctional_corrected/of_v_clonotypes_TRB.csv', index = False)
of_trbj_matrix.to_csv('../../../data/cohort_I/onlyfunctional_corrected/of_j_clonotypes_TRB.csv', index = False)

In [ ]:
base_path = '../../../data/cohort_I/nonfunctional/'#'/home/obagrova/TRB_onlyfunc_nonfunc/nonfunctional/data/'
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=new_meta,
                                 threads=28,
                                 gene = 'TRB',
                                 paths=[base_path + r['run']
                                        for _, r in new_meta.iterrows()])
print(time.time() - t0)

In [ ]:
dataset.metadata

In [ ]:
dataset.evaluate_segment_usage()

In [ ]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [ ]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')


In [ ]:
dataset[0].number_of_reads

In [ ]:
norm_usage_table_v

#### Sampling reads from the repertoire, not changing the gene usage distribution

In [ ]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [ ]:
sampled_repertoire

#### Resampling the repertoire using the new derived usages for V and J genes

In [ ]:
dataset[0]

In [ ]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

In [ ]:
resampled_repertoire

#### Resampling all the reperoires from a dataset

In [ ]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)

In [ ]:
dataset[14]

In [ ]:
resampled_dataset[14]

In [ ]:
nf_trbv_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='V').segment_usage_matrix

In [ ]:
nf_trbj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='J').segment_usage_matrix

In [ ]:
nf_trbj_matrix

In [ ]:
nf_trbv_matrix.to_csv('../../../data/cohort_I/nonfunctional_corrected/nf_v_clonotypes_TRB.csv', index = False)
nf_trbj_matrix.to_csv('../../../data/cohort_I/nonfunctional_corrected/nf_j_clonotypes_TRB.csv', index = False)